<center><font color=#555555><font size=6> MAF1731N - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Science Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> Algorithmic Trading for Market Making </font> <br> <br>

<center><font color=#555555><font size=5> Enriquez Nares Diego Emilio, <br> Sánchez Pazko José Walden, <br> Cuevas Mendoza Carlos </font> <br> <br>

<center> <font color= #555555> <font size = 4> June 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/waldensanchez/myst-team_2-proyect' target="_blank">Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Module 3 </font> <br> <br> <font color= #555555> <font size = 5> Algorithmic Trading for Market Making </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  In this document we are going to see how a **Market Making** (_WBTC/USDT_) extracted from <a href='https://info.uniswap.org/#/pools/0x99ac8ca7087fa4a2a1fb6357269965a2014abc35' target="_blank"> Uniswap</a> works. We are going to follow _several steps_. First, we are going to _install and load all the packages and dependencies_ that we need to run the data we are working on. Second, we are going to _read and visualize the data_ to observe the values we are working with. Finally, we are going to _perform all the processes and display the results_ of the **Automated Market Making Bot** (and all the data before this final process). </font>

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.02in solid gray"> </hr>

This notebook is provided as a case of study of Algorithmic Trading for Market Making. Data within this notebook was extracted from the Bitfinex exchange.

<br>

# <font color= #6B6B6B> 2. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

## <font color= #6B6B6B> 2.1 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas>=1.1.0
- numpy>=1.19.1
- jupyter>=1.0.0
- json

## <font color= #6B6B6B> 2.2 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

files/orderbooks_05jul21.json: JSON file with Data from the Bitfinex exchange.

## <font color= #6E6E6E> 2.3 Install Packages </font>

In [1]:
%%capture

# Install all the pip packages in the requirements.txt
import os
import sys
!{sys.executable} -m pip install -r requirements.txt

## <font color= #6E6E6E> 2.4 Load Packages </font>

In [2]:
# -- Generic
import pandas as pd
import numpy as np
import json

## <font color= #6E6E6E> 2.5 Load Other Scripts </font>

In [3]:
# -- Project scripts
import functions as fn
import data as dt

<br>

# <font color= #6B6B6B> 3. Read and Visualize Data </font>

<hr style="border:0.02in solid gray"> </hr>

Unsiwap data is stored in a **JSON** formated file, which stands for **J**ava **S**cript **O**bject **N**otation, in practical terms, it is very similar to a python dictionary since it has a list-like structure of storing the data, with one or several keys and each of which can have more keys and or data points.

Remember also that a Pandas DataFrame is also in its core form a dictionary where each colum is a key and the column contents are each key content So in way, a **JSON** file is one option to represent a multidimension DataFrame.



## <font color= #6B6B6B> 3.1 Load and display the data </font>

To read the contents of the file is very simple, by using the library _json_, it is a pre-installed one and you do need to import it but its already installed in the python base installation.

In [4]:
# Returns JSON object as a dict (timestamps as keys & dicts as values)
data=dt.read_data(file_name='orderbooks_05jul21.json',
                  folder_route='files/')

In [5]:
# Display the keys of 'data'
data.keys()

dict_keys(['bitfinex', 'kraken'])

In [6]:
# Display the length of 'data['bitfinex']'
len(data['bitfinex'])

3602

<br>

# <font color= #6B6B6B> 4. Pocesses </font>

<hr style="border:0.02in solid gray"> </hr>

In [7]:
# ----- Market information
# Data Frame with information about BTC/USDT orderbooks from July 2021.
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()

## <font color= #6B6B6B> 4.1 Resample </font>

In [8]:
# Resampled Data Frame with information each second
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('S').last().ffill()

## <font color= #6B6B6B> 4.2 TOB </font>

In [9]:
# Getting only the Top Of the Book of each Order Book.
df_bitfinex_tob=fn.data_tob_resample(ob=bitfinex_tob)for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

## <font color= #6B6B6B> 4.3 Useful Data </font>

In [10]:
# Returns INFO objects as dataframes
df_info, df_inventory, df_rebalance = fn.dataframes()

# SLA Conditions: Parameters taken from the market making contract and the trading platform.
contracted_volume = .0001
platform_fee = .000025

## <font color= #6B6B6B> 4.4 Automated Market Making Bot </font>

In [ ]:
# Market Making Bot
df_orders = pd.DataFrame([
    [df_bitfinex_tob.iloc[0]['timestamp'], df_bitfinex_tob.iloc[0]['timestamp']],
    [1,2], ['buy','sell'], [df_bitfinex_tob.iloc[0]['bid'], df_bitfinex_tob.iloc[0]['ask']],
    [contracted_volume, contracted_volume], ['BTC/USDT','BTC/USDT']
    ], 
    index =  ['timestamp','order_id','side','price', 'order_amount','symbol'],
    columns = [0,1]
).transpose()

for side_filled in ['ask','bid']:
    filled = fn.filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == fn.names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )
    if filled > 0:            
            df_trades = fn.execute_trade(
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                side = side_filled,
                i = 1,
                passed_index = 0, 
                current_id = 1, 
                traded_volume = filled,
                trade_fee = platform_fee,
                first_input = True)
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            fn.inventory(
                 index_inventory = passed_index_inventory,
                 df_trades = df_trades, 
                 df_inventory = df_inventory
                 )

            if len(df_rebalance) == 0:
                index_rebalance = 0
            else:
                index_rebalance = df_rebalance.index[-1] + 1
            rebalance(
                trade_fee = platform_fee,
                rebalance_index = index_rebalance,
                df_inventory = df_inventory, 
                df_rebalance = df_rebalance,
                df_trades = df_trades
                )
            # ----

            last_id = df_orders.iloc[-1]['order_id']
            fn.place_order(
                i = 1,
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                passed_index = 2,
                side = side_filled,
                current_id = last_id + 1, 
                amount = contracted_volume
                )
            
for period in range(2,len(df_bitfinex_tob)):
    for side_filled in ['ask','bid']:
        filled = fn.filled_volume(
            size_variation = df_bitfinex_tob.iloc[period][side_filled+'_size'] - df_bitfinex_tob.iloc[period - 1][side_filled+'_size'],
            operation = side_filled, 
            price_variation = df_bitfinex_tob.iloc[period][side_filled] - df_orders[df_orders['side'] == fn.names(side_filled)].iloc[-1]['price'], 
            constant_position_base = contracted_volume
        )
        if filled > 0:
            try:
                last_id_trades = df_trades.iloc[-1]['trade_id']
                passed_index_trades = df_trades.index[-1] + 1
                fn.execute_trade(
                    df_bitfinex_tob = df_bitfinex_tob,
                    df_trades = df_trades,
                    df_orders = df_orders,
                    side = side_filled,
                    i = period,
                    passed_index = passed_index_trades, 
                    current_id = last_id_trades + 1, 
                    traded_volume = filled, 
                    trade_fee = platform_fee
                    )
            except:
                df_trades = fn.execute_trade(
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                side = side_filled,
                i = 1,
                passed_index = 0, 
                current_id = 1, 
                traded_volume = filled,
                trade_fee = platform_fee,
                first_input = True)
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            fn.inventory(
                 index_inventory = passed_index_inventory,
                 df_trades = df_trades, 
                 df_inventory = df_inventory
                 )
            
            if len(df_rebalance) == 0:
                index_rebalance = 0
            else:
                index_rebalance = df_rebalance.index[-1] + 1
            rebalance(
                trade_fee = platform_fee,
                rebalance_index = index_rebalance,
                df_inventory = df_inventory, 
                df_rebalance = df_rebalance,
                df_trades = df_trades
                )
            # ----

            last_id_orders = df_orders.iloc[-1]['order_id']
            passed_index_orders = df_orders.index[-1] + 1
            fn.place_order(
                i = period,
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                passed_index = passed_index_orders,
                side = side_filled,
                current_id = last_id_orders + 1, 
                amount = contracted_volume
                )

<br>

# <font color= #6B6B6B> <font size = 6> 5. Results </font>

<hr style="border:0.02in solid gray"> </hr>

## <font color= #6B6B6B> 5.1 Results of TOB </font>

In [1]:
# Display the first 5 rows & last 5 rows of 'data_bitfinex'
display(df_bitfinex_tob.head(5),
        df_bitfinex_tob.tail(5))

NameError: name 'df_bitfinex_tob' is not defined

## <font color= #6B6B6B> 5.2 TOB Resample </font>

In [15]:
# Display de shape of 'df_bitfinex_tob'
df_bitfinex_tob.shape

(61, 5)

In [16]:
# Display the first 5 rows & last 5 rows of 'df_bitfinex_tob'
display(df_bitfinex_tob.head(5),
        df_bitfinex_tob.tail(5))

,timestamp,bid_size,bid,ask,ask_size
0,2021-07-05 13:06:00+00:00,0.338172,28282.0,28283.0,0.100600
1,2021-07-05 13:07:00+00:00,0.594797,28323.0,28325.0,2.792849
2,2021-07-05 13:08:00+00:00,0.093481,28309.0,28315.0,0.403360
3,2021-07-05 13:09:00+00:00,0.339650,28329.0,28333.0,0.148733
4,2021-07-05 13:10:00+00:00,3.807731,28336.0,28341.0,0.003524


,timestamp,bid_size,bid,ask,ask_size
56,2021-07-05 14:02:00+00:00,0.446627,28288.0,28292.0,3.550600
57,2021-07-05 14:03:00+00:00,0.600127,28295.0,28301.0,0.385010
58,2021-07-05 14:04:00+00:00,0.357682,28340.0,28352.0,0.009931
59,2021-07-05 14:05:00+00:00,0.006000,28355.0,28361.0,0.445649
60,2021-07-05 14:06:00+00:00,0.007629,28354.0,28359.0,0.003464


## <font color= #6B6B6B> 5.3 Useful Dataframes </font>

In [17]:
# Display the dataframe 'df_info'
display(df_info)

,bid_volume,ask_volume,commission
0,0.0001,0.0001,0.000025


,timestamp,order_id,side,price,order_amount,symbol


## <font color= #6B6B6B> 5.4 Automated Market Making Bot </font>

In [ ]:
# Positioned Orders
df_orders

In [ ]:
# Trades Performed
df_trades

In [ ]:
# Liquidity Pool
df_inventory

In [ ]:
# Rebalance History
df_rebalance

<br>

# <font color= #6B6B6B> <font size = 6> 6. Conclusions </font>

In conclusion, based on the analysis presented in this document, it can be concluded that the results displayed by the Automated Market Making Bot represent the actual trades executed in relation to the published orders on the Bitfinex Exchange. Additionally, we can conclude that it is complex to fulfill the tasks that an Exchange must perform in order to maintain a fair structure for all participants.

# <font color= #6B6B6B> <font size = 6> 7. References </font>

<hr style="border:0.02in solid gray"> </hr>

[1] Munnoz, 2020. Python project template. https://github.com/iffranciscome/python-project. (2021).

<br>